### Importing the libraries 

In [7]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os
import time

In [8]:
# Loading environment variables from .env file
load_dotenv()

# Getting the API key from environment variables
api_key = os.getenv('IATI_API_KEY')

In [9]:
# # Loading environment variables from .env file
# load_dotenv()

# # Getting the API key from environment variables
# api_key = os.getenv('IATI_API_KEY')

# # API endpoint
# url = 'https://api.iatistandard.org/datastore/activity/iati'

# # parameters 
# params = {
#     'q': '*:*',  # Query all records
#     'rows': 5 ,   # Limit the number of results
#     'fq': 'iati_identifier:"CH-4-1980001679"',    # Filter for specific identifier
# }

# # headers
# headers = {
#     'Accept': 'application/json',
#     'Ocp-Apim-Subscription-Key': api_key 
# }

# # Making the GET request with parameters and headers
# response = requests.get(url, params=params, headers=headers)

# # response content for debugging
# print(response.content)

# if response.status_code == 200:
#     try:
#         # Parsing the XML response
#         root = ET.fromstring(response.content)
        
#         # Extracting data from XML
#         activities = []
#         for activity in root.findall('.//iati-activity'):
#             activity_data = {
#                 'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
#                 'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
#                 'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
#                 'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
#                 'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
#                 'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
#                 'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
#                 'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
#                 'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
#                 'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
#                 'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
#                 'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
#                 'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
#                 'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
#                 'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
#                 'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
#                 'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
#             }
#             activities.append(activity_data)
        
#         # Converting to DF
#         df = pd.DataFrame(activities)
#         display(df)
#     except ET.ParseError as e:
#         print("Error parsing XML:", e)
# else:
#     print(f"Error: {response.status_code}")


pagination, since even if i delete row limitation on the code above I get max 10 rows.

In [10]:
# # Load environment variables from .env file
# load_dotenv()

# # Get the API key from environment variables
# api_key = os.getenv('IATI_API_KEY')

# # API endpoint
# url = 'https://api.iatistandard.org/datastore/activity/iati'

# # Function to fetch data with pagination
# def fetch_data(api_key, url, rows_per_page=100, max_pages=10):
#     all_activities = []
#     start = 0
#     for page in range(max_pages):
#         # Parameters with pagination
#         params = {
#             'q': '*:*',  # Query all records
#             'rows': rows_per_page,  # Number of results per page
#             'start': start,  # Starting point for each page
#             'fq': [
#                 'iati_identifier:"CH-4-1980001679"', 
#                 #'reporting_org_ref:"CH-4"'  
#             ]
#         }

#         # Headers
#         headers = {
#             'Accept': 'application/json',
#             'Ocp-Apim-Subscription-Key': api_key 
#         }

#         # Making the GET request with parameters and headers
#         response = requests.get(url, params=params, headers=headers)

#         if response.status_code == 200:
#             try:
#                 # Parsing the XML response
#                 root = ET.fromstring(response.content)

#                 # Extracting data from XML
#                 activities = []
#                 for activity in root.findall('.//iati-activity'):
#                     activity_data = {
#                         'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
#                         'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
#                         'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
#                         'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
#                         'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
#                         'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
#                         'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
#                         'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
#                         'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
#                         'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
#                         'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
#                         'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
#                         'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
#                         'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
#                         'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
#                         'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
#                         'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
#                     }
#                     activities.append(activity_data)

#                 all_activities.extend(activities)

#                 # Increment the start for the next page
#                 start += rows_per_page

#                 # Break if no more results
#                 if len(activities) < rows_per_page:
#                     break

#             except ET.ParseError as e:
#                 print("Error parsing XML:", e)
#                 break
#         else:
#             print(f"Error: {response.status_code}")
#             break

#     return all_activities

# # Fetching data
# activities = fetch_data(api_key, url, rows_per_page=100, max_pages=10)

# # Converting to DataFrame
# df_ch4 = pd.DataFrame(activities)


In [11]:
# load_dotenv()


# api_key = os.getenv('IATI_API_KEY')

# # API endpoint
# url = 'https://api.iatistandard.org/datastore/activity/iati'

# # Function to fetch data with pagination
# def fetch_data(api_key, url, rows_per_page=100, max_pages=10):
#     all_activities = []
#     start = 0
#     for page in range(max_pages):
#         # Parameters with pagination
#         params = {
#             'q': '*:*',  
#             'rows': rows_per_page,  
#             'start': start, 
#             # 'fq': [
#             #     'iati_identifier:"CH-4-1980001679"',  
#             #     'reporting_org_ref:"CH-4"'  
#             # ]
#         }

#         # Headers
#         headers = {
#             'Accept': 'application/json',
#             'Ocp-Apim-Subscription-Key': api_key 
#         }

#         # Making the GET request with parameters and headers
#         response = requests.get(url, params=params, headers=headers)

#         if response.status_code == 200:
#             try:
#                 # Parsing the XML response
#                 root = ET.fromstring(response.content)

#                 # Extracting data from XML
#                 activities = []
#                 for activity in root.findall('.//iati-activity'):
#                     activity_data = {
#                         'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
#                         'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
#                         'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
#                         'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
#                         'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
#                         'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
#                         'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
#                         'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
#                         'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
#                         'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
#                         'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
#                         'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
#                         'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
#                         'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
#                         'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
#                         'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
#                         'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
#                     }
#                     activities.append(activity_data)

#                 all_activities.extend(activities)

#                 # Increment the start for the next page
#                 start += rows_per_page

#                 # Break if no more results
#                 if len(activities) < rows_per_page:
#                     break

#             except ET.ParseError as e:
#                 print("Error parsing XML:", e)
#                 break
#         else:
#             print(f"Error: {response.status_code}")
#             break

#     return all_activities


# activities = fetch_data(api_key, url, rows_per_page=100, max_pages=10)

# # Converting to DataFrame
# df = pd.DataFrame(activities)


### Adding delay

In [12]:
# def fetch_data(api_key, url, rows_per_page=100, max_pages=10, delay=1):
#     all_activities = []
#     start = 0
#     for page in range(max_pages):
#         # Parameters with pagination and filter query
#         params = {
#             'q': '*:*',  
#             'rows': rows_per_page, 
#             'start': start,  
#             'fq': [
#                 'reporting_org_ref:"CH-4"'  
#             ]
#         }

#         # Headers
#         headers = {
#             'Accept': 'application/json',
#             'Ocp-Apim-Subscription-Key': api_key 
#         }

#         while True:
#             # Making the GET request with parameters and headers
#             response = requests.get(url, params=params, headers=headers)

#             if response.status_code == 200:
#                 try:
#                     # Parsing the XML response
#                     root = ET.fromstring(response.content)

#                     # Extracting data from XML
#                     activities = []
#                     for activity in root.findall('.//iati-activity'):
#                         activity_data = {
#                             'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
#                             'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
#                             'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
#                             'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
#                             'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
#                             'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
#                             'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
#                             'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
#                             'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
#                             'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
#                             'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
#                             'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
#                             'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
#                             'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
#                             'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
#                             'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
#                             'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
#                         }
#                         activities.append(activity_data)

#                     all_activities.extend(activities)

#                     # Increment the start for the next page
#                     start += rows_per_page

#                     # Break if no more results
#                     if len(activities) < rows_per_page:
#                         break

#                     # Break the while loop to move to the next page
#                     break

#                 except ET.ParseError as e:
#                     print("Error parsing XML:", e)
#                     break
#             elif response.status_code == 429:
#                 retry_after = int(response.headers.get("Retry-After", delay))
#                 print(f"Rate limited. Retrying after {retry_after} seconds...")
#                 time.sleep(retry_after)
#             else:
#                 print(f"Error: {response.status_code}")
#                 break

#     return all_activities

# # Fetch data with the specified parameters
# activities = fetch_data(api_key, url, rows_per_page=100, max_pages=30, delay=1)

# # Converting to DataFrame
# df = pd.DataFrame(activities)

# # Display the DataFrame
# print(df)


Updated code to get all the results 

In [13]:
# Defining API endpoint
url = 'https://api.iatistandard.org/datastore/activity/iati'
params = {
    'q': '*:*',  # Query all records
    'fq': 'reporting_org_ref:"CH-4"', #filtering the sector code 
    'rows': 100,  # Number of rows per page
    'start': 0   # Starting row number
}

# Headers
headers = {
    'Accept': 'application/xml',  # Accept XML
    'Ocp-Apim-Subscription-Key': api_key
}

all_data = []
retry_attempts = 5  # Number of times to retry after a 429 response

def extract_activity_data(activity):
    """Extract data from an activity element into a dictionary."""
    activity_data = {}
    for elem in activity.iter():
        if elem.tag != activity.tag:  
            # I Used element tag as key,text as value
            if elem.text and elem.text.strip():
                activity_data[elem.tag] = elem.text.strip()
            # Adding attributes as additional key-value pairs
            for attr_name, attr_value in elem.attrib.items():
                activity_data[f"{elem.tag}_{attr_name}"] = attr_value
    return activity_data

while True:
    response = requests.get(url, params=params, headers=headers)
    print(f"Status Code: {response.status_code}, Start: {params['start']}")

    if response.status_code == 200:
        try:
            # Parsing XML
            root = ET.fromstring(response.content)
            activities = root.findall('iati-activity')
            
            # If no more activities are returned, break the loop
            if not activities:
                break
            
            # Extracting data
            for activity in activities:
                activity_data = extract_activity_data(activity)
                all_data.append(activity_data)
            
            # Incrementing the start parameter for pagination
            params['start'] += params['rows']
        except ET.ParseError as e:
            print(f"Error parsing XML: {e}")
            break
    elif response.status_code == 429:
        print("Rate limit exceeded. Waiting before retrying...")
        time.sleep(60)  # Waiting for 60 seconds before retrying
        retry_attempts -= 1
        if retry_attempts <= 0:
            print("Max retry attempts reached. Exiting...")
            break
    else:
        print(f"Error: {response.status_code}")
        break
        
# Converting to DataFrame
df_ch_4 = pd.DataFrame(all_data)
df_ch_4.head()



Status Code: 200, Start: 0
Status Code: 429, Start: 100
Rate limit exceeded. Waiting before retrying...
Status Code: 200, Start: 100
Status Code: 429, Start: 200
Rate limit exceeded. Waiting before retrying...
Status Code: 200, Start: 200
Status Code: 429, Start: 300
Rate limit exceeded. Waiting before retrying...
